In [2]:
from dmrApp import db
from dmrApp.models import Restaurants, Employees, Dmrs, Employeetypes, User, Shifts
from datetime import date
from sqlalchemy import func, desc

In [3]:
dateFrom=date(2020,12,21)
dateTo=date(2020,12,22)

In [19]:
dmrDataReport= db.session.query(Dmrs, func.max(Dmrs.id)).filter(Dmrs.dmrDate>=dateFrom, Dmrs.dmrDate<=dateTo).group_by(Dmrs.restaurantId, Dmrs.dmrDate).all()
dmrDateList = [i[0].dmrDate for i in dmrDataReport]
restaurantList = [i[0].restaurant.name for i in dmrDataReport]
startCashList = [i[0].startCash for i in dmrDataReport]
payoutList = [i[0].payout for i in dmrDataReport]
salesList = [i[0].sales for i in dmrDataReport]




In [15]:
import pandas as pd

In [20]:
dfDmrs = pd.DataFrame(list(zip(dmrDateList,restaurantList, startCashList,payoutList,salesList)),
                     columns=['date','restaurant','startCash','payout','sales'])

dfDmrs

,date,restaurant,startCash,payout,sales
0,2020-12-21,Roll Bar,1.0,NaN,1.0
1,2020-12-21,Chalkboard,NaN,1.0,NaN


In [8]:
startCashList

[1.0]

In [10]:
def shiftDataLists(shiftData):
    shiftNamesList = []
    for i in shiftData.all():
        shiftNamesList.append(i.employee.name)
    shiftRolesList = []
    for i in shiftData.all():
        shiftRolesList.append(
            Employeetypes.query.filter_by(id=i.employeeTypesId).with_entities(Employeetypes.role).first()[0])

    shiftSchedTime = shiftData.with_entities(Shifts.schedTime).all()
    shiftSchedTimeList = [i[0] for i in shiftSchedTime]
    shiftTimeStart = shiftData.with_entities(Shifts.timeStart).all()
    shiftTimeStartList = [i[0].strftime("%I:%M %p") for i in shiftTimeStart]
    shiftTimeOff = shiftData.with_entities(Shifts.timeOff).all()
    shiftTimeOffList = [i[0].strftime("%I:%M %p") for i in shiftTimeOff]
    shiftHoursWorked = shiftData.with_entities(Shifts.hoursWorked).all()
    shiftHoursWorkedList = ["{:.1f}".format(i[0]) for i in shiftHoursWorked]
    shiftTips = shiftData.with_entities(Shifts.shiftTipsShipgarten).all()
    shiftTipList = ["{:.2f}".format(i[0]) for i in shiftTips]
    shiftCount = len(shiftData.all())
    shiftRestaurants = shiftData.all()
    shiftRestaurantList = [i.restaurant.name for i in shiftRestaurants]

    return (shiftRolesList,shiftRestaurantList,shiftNamesList,shiftSchedTimeList,
            shiftTimeStartList,shiftTimeOffList,shiftHoursWorkedList,
            shiftTipList)

In [11]:
shiftData = Shifts.query.filter(Shifts.shiftDate>=dateFrom,
                                Shifts.shiftDate<=dateTo)
shiftReportColumns = list(shiftDataLists(shiftData))

In [12]:
shiftReportColumns

[[], [], [], [], [], [], [], []]